In [44]:
import pandas as pd
import string
import re
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
import gensim
import matplotlib.pyplot as plt
import urllib.parse


In [49]:
file1 = pd.read_csv("data/the_guardian2019-10-24_first_part.csv")
file2 = pd.read_csv("data/the_guardian2019-10-24_second_part.csv")
file3 = pd.read_csv("data/the_guardian2019-10-25_third_part.csv")
file4 = pd.read_csv("data/the_guardian2019-10-25_fourth_part.csv")
mergedGuardians = pd.merge( file3, file4 ,on=['headline','standfirst', 'author', 'date_time','content', 'url'],how = 'outer')
mergedGuardians = pd.merge( file1, mergedGuardians ,on=['headline','standfirst', 'author', 'date_time','content', 'url'],how = 'outer')
mergedGuardians = pd.merge( mergedGuardians, file2,on=['headline','standfirst', 'author', 'date_time','content', 'url'],how = 'outer')
mergedGuardians.dropna( inplace = True )
null_columns = mergedGuardians.columns[mergedGuardians.isnull().any()]
print(mergedGuardians[mergedGuardians.isnull().any(axis=1)][null_columns])
mergedGuardians.index = mergedGuardians.index + 1
mergedGuardians

Empty DataFrame
Columns: []
Index: []


,headline,standfirst,author,date_time,content,url
1,More US firms than ever get transparent over p...,Protests against political associations have p...,Edward Helmore,2019-10-24T13:00:49+0100,Fearful of facing public wrath in a heated pol...,https://www.theguardian.com/us-news/2019/oct/2...
2,Brexit 'do or die': what are Boris Johnson's o...,Will the PM keep trying to push his withdrawal...,Heather Stewart,2019-10-24T12:58:43+0100,Boris Johnson’s 31 October “do or die” Brexit ...,https://www.theguardian.com/politics/2019/oct/...
3,Chinese orienteering team disqualified for che...,Athletes lose moral compass after using secret...,Lily Kuo,2019-10-24T12:27:17+0100,A Chinese orienteering team has been disqualif...,https://www.theguardian.com/world/2019/oct/24/...
4,Five conflict photographers on some of the har...,The new book Conversations on Conflict Photogr...,Jehan Jillani,2019-10-24T12:00:11+0100,In the newly published Conversations on Confli...,https://www.theguardian.com/world/2019/oct/24/...
5,Venice tourist 'kidnaps' currency exchange wor...,Woman reportedly held hostage in latest incide...,Angela Giuffrida,2019-10-24T11:47:55+0100,A tourist is facing charges of kidnapping afte...,https://www.theguardian.com/world/2019/oct/24/...
7,Nusrat Jahan Rafi: 16 sentenced to death over ...,Student set on fire after refusing to withdraw...,Peter Beaumont,2019-10-24T11:33:29+0100,Sixteen Bangladeshis have been sentenced to de...,https://www.theguardian.com/world/2019/oct/24/...
8,Sri Lanka bombings: spy chief lambasted in dam...,Investigation finds series of intelligence and...,Michael Safi,2019-10-24T11:29:14+0100,A damning Sri Lankan parliamentary investigati...,https://www.theguardian.com/world/2019/oct/24/...
9,Germany downturn drags eurozone to near stagna...,Jobs slump and year-long fall in output push E...,Phillip Inman,2019-10-24T11:20:14+0100,German companies shed more jobs this month tha...,https://www.theguardian.com/business/2019/oct/...
10,Scroll depicting early life of Buddha goes on ...,Burmese manuscript is among 120 items at Briti...,Mark Brown,2019-10-24T11:10:49+0100,A vibrantly coloured 19th-century Burmese illu...,https://www.theguardian.com/books/2019/oct/24/...
11,TVs sold by Amazon and Best Buy 'contain chemi...,Several TV lines on sale in the US contain fla...,Oliver Milman,2019-10-24T11:01:07+0100,Popular brands of televisions sold by Amazon a...,https://www.theguardian.com/us-news/2019/oct/2...


In [62]:
def processing( dates ):
    dates = dates.replace('T', '/ ')
    dates = dates.replace('+0100','')
    return dates

mergedGuardians['date_time'] = mergedGuardians['date_time'].apply(processing)


#mergedGuardians.drop('url',axis = 1)
cols = ['author', 'content','headline', 'standfirst', 'date_time']
mergedGuardians = mergedGuardians[cols]
mergedGuardians.rename( columns = { 'date_time' : 'time' }, inplace = True )
mergedGuardians

C:\Users\LM\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,author,content,headline,standfirst,time
1,Edward Helmore,Fearful of facing public wrath in a heated pol...,More US firms than ever get transparent over p...,Protests against political associations have p...,2019-10-24/13:00:49
2,Heather Stewart,Boris Johnson’s 31 October “do or die” Brexit ...,Brexit 'do or die': what are Boris Johnson's o...,Will the PM keep trying to push his withdrawal...,2019-10-24/12:58:43
3,Lily Kuo,A Chinese orienteering team has been disqualif...,Chinese orienteering team disqualified for che...,Athletes lose moral compass after using secret...,2019-10-24/12:27:17
4,Jehan Jillani,In the newly published Conversations on Confli...,Five conflict photographers on some of the har...,The new book Conversations on Conflict Photogr...,2019-10-24/12:00:11
5,Angela Giuffrida,A tourist is facing charges of kidnapping afte...,Venice tourist 'kidnaps' currency exchange wor...,Woman reportedly held hostage in latest incide...,2019-10-24/11:47:55
7,Peter Beaumont,Sixteen Bangladeshis have been sentenced to de...,Nusrat Jahan Rafi: 16 sentenced to death over ...,Student set on fire after refusing to withdraw...,2019-10-24/11:33:29
8,Michael Safi,A damning Sri Lankan parliamentary investigati...,Sri Lanka bombings: spy chief lambasted in dam...,Investigation finds series of intelligence and...,2019-10-24/11:29:14
9,Phillip Inman,German companies shed more jobs this month tha...,Germany downturn drags eurozone to near stagna...,Jobs slump and year-long fall in output push E...,2019-10-24/11:20:14
10,Mark Brown,A vibrantly coloured 19th-century Burmese illu...,Scroll depicting early life of Buddha goes on ...,Burmese manuscript is among 120 items at Briti...,2019-10-24/11:10:49
11,Oliver Milman,Popular brands of televisions sold by Amazon a...,TVs sold by Amazon and Best Buy 'contain chemi...,Several TV lines on sale in the US contain fla...,2019-10-24/11:01:07


In [64]:
mergedGuardians.to_csv('data/merged_cleaned_TheGuardians.csv', encoding='utf-8-sig')